In [33]:
import cv2 as cv
import numpy as np
import math

map_init = cv.imread("assets/unlabelled_map.png", cv.IMREAD_COLOR)
#map_init = cv.imread("assets/unlabelled_map.png", cv.IMREAD_GRAYSCALE)
cv.imshow("Unprocessed Map", map_init)
cv.waitKey(0)
cv.destroyAllWindows()

In [8]:
bin_map_nodfs = np.full((476,890), 0, dtype=np.uint8)
for i in range(476):
    for j in range(890):
        if(tuple(map_init[i,j]) == (255, 255, 255)):
            bin_map_nodfs[i,j] = 255

cv.imshow("Binary unprocessed map, before DFS", bin_map_nodfs)
cv.waitKey(0)
cv.destroyAllWindows()

In [3]:
#Taking the Final Roadmap as the current binary Image
bin_map = bin_map_nodfs

### RRT* Implementation

In [38]:
class Graph:
    def __init__(self, display):
        self.__AdjList = []
        self.Coordinates = []
        self.image = display
        self.maxminx = None
        self.maxminy = None
        print("New Graph has been created")#debug

    def add_node(self, coordinates):
        self.__AdjList.append([])
        self.Coordinates.append(coordinates)
        y, x = coordinates
        print("New coordinates", coordinates, "have been added")#debug
        cv.circle(self.image, (x, y), 2, color=(255,0,0))
        if(self.maxminx == None):
            self.maxminx = (x, x)
            self.maxminy = (y, y)
        else:
            self.maxminx = (max(self.maxminx[0], x), min(self.maxminx[1], x))
            self.maxminy = (max(self.maxminy[0], y), min(self.maxminy[1], y))

    def add_edge(self, i, j):
        self.__AdjList[i].append(j)
        self.__AdjList[j].append(i)
        y1, x1 = self.Coordinates[i]
        y2, x2 = self.Coordinates[j]
        cv.line(self.image, (x1, y1), (x2, y2), color=(0,255,0), thickness=1)

    def sample_point(self):
        range = 10
        '''
        x = np.random.randint(max(self.maxminx[1]-range, 0), min(self.maxminx[0]+range, 890))
        y = np.random.randint(max(self.maxminy[1]-range, 0), min(self.maxminy[0]+range, 476))
        '''
        x, y = np.random.randint(890), np.random.randint(476)
        print("sampled point: ", (y, x)) #debug
        return (y, x)

    def find_Nearest(self, coordinates):
        found_a_reachable_point = False
        min_dist = math.sqrt(475*475 + 889*889)
        y1, x1 = coordinates
        for i in range(len(self.Coordinates)):
            y2, x2 = self.Coordinates[i]
            dist = math.sqrt((y2-y1)**2+(x2-x1)**2)
            if(min_dist > dist):
                empty_image = np.zeros((476, 890), dtype=np.uint8)
                cv.line(empty_image, (x1, y1), (x2, y2), color=255, thickness=1)
                should_be_just_a_line = cv.bitwise_and(empty_image, bin_map)
                if(np.array_equal(should_be_just_a_line, empty_image)):
                    cv.imshow("Empty Image", empty_image)
                    cv.imshow("Bitwise AND", should_be_just_a_line)
                    min_dist = dist
                    nearest = i
                    found_a_reachable_point = True
        if(found_a_reachable_point == True):
            return nearest
        else:
            return None
        
def movement_towards_unit_vector(point1, point2, move_distance):
    y1, x1 = point1
    y2, x2 = point2
    dist = math.sqrt((x2-x1)**2+(y2-y1)**2)
    if(dist>move_distance):
        y = np.floor(y1+(y2-y1)*move_distance/dist + 0.5).astype(np.uint16)
        x = np.floor(x1+(x2-x1)*move_distance/dist + 0.5).astype(np.uint16)
        return y,x
    else:
        return point2

def RRT_star(bin_image, initial_pos, destination, dist_trim, goal_radius):
    temp_image = cv.cvtColor(bin_image, cv.COLOR_GRAY2BGR)
    G = Graph(temp_image)
    G.add_node(initial_pos)
    while True:
        #End this loop when nearest vertex to goal is within 'goal_radius'
        cv.imshow("RRT Path Planning", temp_image)
        print("Image shown")#debug
        Key = cv.waitKey(20)
        if(Key != -1):
            print("Loop broken due to Key Press")
            break
        while True:
            #End this loop when the Tree has grown by one vertex
            point = G.sample_point()
            if(bin_image[point] == 255):
                nearest = G.find_Nearest(point)
                if(nearest == None):
                    continue
                else:
                    y, x = movement_towards_unit_vector(G.Coordinates[nearest], point, dist_trim)
                    G.add_node((y, x))
                    G.add_edge(-1, nearest)
                    break
        y1, x1 = destination
        y2, x2 = G.Coordinates[-1]
        dist = math.sqrt((y2-y1)**2 + (x2-x1)**2)
        print("Node added")
        if(dist <= goal_radius):
            G.add_node(destination)
            G.add_edge(-2, -1)
            print("Goal reached! press any key to exit")
            cv.imshow("RRT Path Planning", temp_image)
            break
    cv.waitKey(0)
    cv.destroyAllWindows()
    

In [39]:
RRT_star(bin_map, (188, 187), (223, 724), 100, 17)

New Graph has been created
New coordinates (188, 187) have been added
Image shown
sampled point:  (260, 539)
sampled point:  (17, 683)
sampled point:  (187, 160)
sampled point:  (474, 289)
sampled point:  (118, 44)
sampled point:  (18, 108)
sampled point:  (451, 641)
sampled point:  (195, 757)
sampled point:  (130, 616)
sampled point:  (269, 396)
sampled point:  (73, 145)
sampled point:  (61, 360)
sampled point:  (437, 534)
sampled point:  (443, 375)
sampled point:  (405, 5)
sampled point:  (367, 426)
sampled point:  (432, 573)
sampled point:  (385, 231)
sampled point:  (396, 807)
sampled point:  (48, 756)
sampled point:  (188, 635)
sampled point:  (135, 278)
sampled point:  (451, 752)
sampled point:  (413, 141)
sampled point:  (442, 308)
sampled point:  (179, 516)
sampled point:  (283, 410)
sampled point:  (9, 375)
sampled point:  (11, 494)
sampled point:  (313, 625)
sampled point:  (206, 596)
sampled point:  (465, 398)
sampled point:  (350, 330)
sampled point:  (358, 229)
sampled poi

In [12]:

# Create two binary images
image1 = np.array([[255, 0, 255],
                   [0, 255, 0],
                   [255, 0, 255]], dtype=np.uint8)

image2 = np.array([[255, 255, 0],
                   [0, 255, 255],
                   [255, 255, 0]], dtype=np.uint8)

# Apply bitwise AND operation
result = cv2.bitwise_and(image1, image2)

# Display the result
print("Image 1:")
print(image1)
print("Image 2:")
print(image2)
print("Result of Bitwise AND:")
print(result)
print("is Image1 the same after the Bitwise Operation?", np.array_equal(image1, result))


Image 1:
[[255   0 255]
 [  0 255   0]
 [255   0 255]]
Image 2:
[[255 255   0]
 [  0 255 255]
 [255 255   0]]
Result of Bitwise AND:
[[255   0   0]
 [  0 255   0]
 [255   0   0]]
is Image1 the same after the Bitwise Operation? False
